<a href="https://colab.research.google.com/github/alpha-davidson/ICTP-Citizen-Science-2023/blob/main/Activity1_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Networks in TensorFlow's Keras


We will use a dense neural network in Keras to solve a simple regression problem. 
## Learning Task
We will construct a dense neural network to **predict the invariant mass of a particle** from its energy, momentum, (and charge).

*Note that this task does not require machine learning. We choose a task with a known mapping to help us create, debug, and tune our first neural network.*

## Dataset
This dataset is a collection of simulated particle events from [Pythia](http://home.thep.lu.se/~torbjorn/Pythia.html). The dataset is a 2D array where each row represents one event from an $e^{-} + p$ (electron-proton) collision. This dataset is comprised _only_ of events where exactly 16 particles are produced from an electron-proton collision. Each particle contains $(p_x,p_y,p_z,E,q)$. Each event is therefore represented by 80 numbers. 

**Advanced activity:** There are more interesting event-wise learning tasks using this dataset. Consider crafting your own learning task and target for this data.



## Computational Notes

If this is your first time in a Jupyter-like environment, please read the following carefully:

 - You are in an active kernel
 - Run each cell with `Shift + Enter`
 - You must execute the cells in the order that you want the code to run
 - In Colab, `Runtime`$\rightarrow$`Change runtime type` allows you to utilize GPUs and TPUs. They are unnecessary here, but will become vital in later exercises.


In [1]:
# import the packages we will be using
import numpy as np
import tensorflow as tf
import pylab as plt
import pandas as pd

2023-05-23 17:20:06.009325: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-23 17:20:06.041826: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-23 17:20:06.042374: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 17:20:06.545393: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Import data from github. Note: in colab, go to Files and refresh to see file.
# Here I use Linux commands within the notebook to pull the data file and rename it.

# If you are not in an environment that supports the following commands, 
#     paste the url below to download the dataset onto your local machine.

"""!wget https://github.com/NuclearTalent/MachineLearningECT/blob/master/doc/pub/Day6/data/homogenous-16-particle-events-energy.npy?raw=true
!mv homogenous-16-particle-events-energy.npy?raw=true particle-events.npy""";

In [3]:
# now we load the data file, which is a numpy array
events = np.load("particle-events.npy")

In [4]:
events.shape

(35916, 80)

In [5]:
df = pd.DataFrame(events)

In [7]:
col_names = [f"px{i}|py{i}|pz{i}|E{i}|q{i}" for i in range(1,17)]
features = []
for col in col_names:
    features = features + col.split("|")
df.columns = features

In [8]:
df

,px1,py1,pz1,E1,q1,px2,py2,pz2,E2,q2,...,px15,py15,pz15,E15,q15,px16,py16,pz16,E16,q16
0,-0.492933,-0.469448,-2.228710,2.334520,-1.0,-0.380699,0.859243,0.125686,0.958388,-1.0,...,0.060035,0.198486,0.387630,0.461235,1.0,-0.182514,0.156148,-1.763670,2.01211,1.0
1,-0.021128,0.513518,0.056984,0.535606,-1.0,0.357259,0.326994,-0.533131,0.733667,-1.0,...,3.329790,-0.872136,4.813310,5.919080,1.0,0.238305,0.223850,-0.542793,1.13220,1.0
2,-0.238883,0.010763,-1.438530,1.464940,-1.0,0.166321,0.511802,0.878539,1.039670,-1.0,...,3.371150,0.149630,12.704600,13.145800,1.0,-0.033907,0.150283,-3.653550,3.77525,1.0
3,0.100541,0.042173,0.535900,0.564407,-1.0,0.481310,-0.186366,0.788642,0.952800,-1.0,...,2.132850,-0.677265,3.723820,4.346730,1.0,-0.321742,-0.034143,-1.928730,2.16911,1.0
4,-3.701960,4.534330,5.796190,8.238930,-1.0,-0.250877,0.350684,-0.728698,0.858137,-1.0,...,0.125843,0.094416,-0.093951,0.230342,1.0,0.624977,-0.087251,-2.548230,2.78784,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35911,-1.333440,1.059830,1.372490,2.191920,-1.0,-0.554874,0.015482,0.105968,0.582094,-1.0,...,-0.181712,0.312801,-0.281528,0.479168,1.0,-0.743300,0.552026,-0.433851,1.38773,1.0
35912,-0.337079,-0.092994,-1.016500,1.083980,-1.0,-0.078552,0.086005,2.134450,2.142170,-1.0,...,0.117765,-0.366370,-0.163362,0.440752,1.0,-0.325950,-0.327742,-1.620970,1.92913,1.0
35913,-0.494778,-0.198240,1.594910,1.687400,-1.0,-0.005706,0.441082,-0.332050,0.569493,-1.0,...,0.274823,-0.141276,-0.151151,0.371232,1.0,-0.776436,-0.033002,-3.395280,3.60725,1.0
35914,-0.550329,0.533369,0.111079,0.786869,-1.0,0.021931,0.324879,0.326723,0.481928,-1.0,...,-0.393085,0.514364,-0.484226,0.820390,1.0,0.382212,-0.248855,-0.179829,1.05863,1.0


Recall that each row of this dataset is an entire event. We need each row to represent a training example, which is a single particle.

Using `numpy`'s `reshape` method we can make each row represent one particle. 

In [9]:
# Here we rearrange the data within each of the events to isolate particles

#print("events[0] =\n",events[0])
evt_particles = np.reshape(events, (len(events), 16, 5))

#print("\nevt_particles[0] =\n", evt_particles[0])

# Use another call of reshape to combine all events to have the appropriate shape
# Complete me:
#particles = 

#print("\nparticles =\n",particles[:10])

These are our training data inputs, but we also must provide the **targets**, which are the **invariant masses of each particle**. This is a straightforward computation.

We choose units where $c = 1$:
$$m^2=E^2-||\textbf{p}||^2$$ 
where $m, E$, and $\textbf{p}$ are all in GeV. This is equivalent to 
$$m = \sqrt{E^2 - (p_x^2 + p_y^2 + p_z^2)}$$

**Create an array of your target values.**
Due to insufficient precision, some $m^2$ values for massless particles will come out very slightly negative. These should be treated as zero to avoid `NaN`. I used the `maximum` method from `numpy` to handle this. 

In [10]:
# Complete me:
masses = pd.DataFrame()
for i in range(16):
    px = np.array(df[f"px{i+1}"]); py = np.array(df[f"py{i+1}"]); pz = np.array(df[f"pz{i+1}"])
    E  = np.array(df[f"E{i+1}"] )
    arg = E**2 - (px**2 + py**2 + pz**2)
    masses[f"m{i+1}"] = np.sqrt( np.maximum(np.zeros(len(arg)), arg) )  

$\rightarrow$ Proton is the heaviest particle created

In [11]:
masses.describe()

,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,m14,m15,m16
count,35916.000000,35916.000000,35916.000000,35916.000000,35916.000000,35916.000000,35916.000000,35916.000000,35916.000000,35916.000000,35916.000000,35916.000000,35916.000000,35916.000000,35916.000000,35916.000000
mean,0.139570,0.139569,0.139571,0.014286,0.001352,0.000749,0.000493,0.000391,0.000395,0.000490,0.000746,0.001369,0.139565,0.139571,0.139571,0.938270
std,0.000490,0.000159,0.000485,0.016553,0.003334,0.001630,0.001183,0.000946,0.000979,0.001164,0.001648,0.003257,0.000846,0.000264,0.000820,0.000012
min,0.129948,0.130457,0.129184,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.126122,0.131046,0.128824,0.938208
25%,0.139565,0.139568,0.139565,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.139556,0.139566,0.139556,0.938264
50%,0.139570,0.139570,0.139570,0.000000,0.000000,0.000000,0.000000,0.000007,0.000000,0.000000,0.000000,0.000007,0.139570,0.139570,0.139570,0.938270
75%,0.139575,0.139572,0.139575,0.028929,0.000848,0.000546,0.000448,0.000403,0.000406,0.000435,0.000547,0.000918,0.139586,0.139574,0.139584,0.938276
max,0.148373,0.146542,0.147937,0.060916,0.049075,0.048046,0.046212,0.033206,0.037981,0.032007,0.040221,0.047000,0.150444,0.148468,0.149858,0.938331


There are several hundred thousand datapoints in this dataset which is overkill for this simple example. Create a test dataset with just 1000 examples.

In [12]:
#Slicing allows you select a subset of an array.
#This can be done like this: smallerArray = largerArray[:100]

complete_data = pd.concat([df,masses], axis=1, join="inner")
data_subset = complete_data.sample(n=1000, replace=False, weights=None, random_state=71)

Next, make a histogram of the target data to make sure that we are seeing masses of real particles. As this data has limited precision, this will not resolve electrons very well, but protons, pions, and massless particles should be clearly visible.

In [13]:
plt.hist(data_subset[masses.columns].stack().tolist(),bins=100)
plt.show()

Now we can build and train the first neural network. We will start with a network with on hidden layer and 5 neurons, and ReLU activation. 




In [25]:
# Build our model
model = tf.keras.Sequential() #Define the model object
model.add(tf.keras.layers.Dense(units=20, # Number of neurons in hidden layer 
                                input_shape=(len(features),), 
                                activation="relu")
         ) #Add a single hidden layer

# Add the output layer yourself
# No activation function assignment defaults to "linear"
# Complete me:
model.add(tf.keras.layers.Dense(units=1, activation="linear"))

# prints a model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 20)                1620      
                                                                 
 dense_3 (Dense)             (None, 1)                 21        
                                                                 
Total params: 1,641
Trainable params: 1,641
Non-trainable params: 0
_________________________________________________________________


To start, train for 30 epochs with a batch size of 128, an Adam optimizer with a learning rate of 0.1, using mean squared error loss. **This is not ideal.**

Define your validation split to provide a set of data the model has not used to assess how the model is generalizing. This allows us to monitor overtraining. Setting `validation_split=0.2` meaning that $20\%$ of the data will be used for validation. **This is not ideal.**

Information on how to implement these features can be found here:
<https://www.tensorflow.org/api_docs/python/tf/keras>.
Check out Sequential underneath models and Dense under layers.

In [26]:
optimize = tf.keras.optimizers.Adam(learning_rate=0.1) # Adam optimizer with a learning rate of 0.1
loss_func = tf.keras.losses.MeanSquaredError() # choosing MSE

# compile model
model.compile(optimizer=optimize,loss=loss_func) 

In [27]:
X = data_subset.drop(masses.columns, axis=1)
X.shape

(1000, 80)

In [28]:
y = data_subset[masses.columns]
y.shape

(1000, 16)

In [29]:
results = model.fit(X, y, epochs=10, batch_size=256, validation_split=0.2)

Epoch 1/10
4/4 [==============================] - 0s 28ms/step - loss: 188.0746 - val_loss: 0.1400
Epoch 2/10
4/4 [==============================] - 0s 7ms/step - loss: 0.1376 - val_loss: 0.1298
Epoch 3/10
4/4 [==============================] - 0s 7ms/step - loss: 0.1427 - val_loss: 0.2336
Epoch 4/10
4/4 [==============================] - 0s 7ms/step - loss: 0.2535 - val_loss: 0.2768
Epoch 5/10
4/4 [==============================] - 0s 9ms/step - loss: 0.2372 - val_loss: 0.2100
Epoch 6/10
4/4 [==============================] - 0s 7ms/step - loss: 0.2209 - val_loss: 0.2526
Epoch 7/10
4/4 [==============================] - 0s 7ms/step - loss: 0.2502 - val_loss: 0.2194
Epoch 8/10
4/4 [==============================] - 0s 7ms/step - loss: 0.2094 - val_loss: 0.2005
Epoch 9/10
4/4 [==============================] - 0s 8ms/step - loss: 0.1958 - val_loss: 0.1898
Epoch 10/10
4/4 [==============================] - 0s 6ms/step - loss: 0.1784 - val_loss: 0.1624


A learning curve is a clearer way to visualize the above training. This helps us determine over and underfitting, which, in turn, helps us tune our model and training.

In [30]:
# helper function for easy plotting
def plot_learning_curve(history):
    plt.plot(history["loss"], label="training loss")
    plt.plot(history["val_loss"], label="validation loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.yscale('log')
    plt.legend()
    plt.grid(alpha=0.5)
    plt.show()

In [31]:
# using the new function
plot_learning_curve(results.history)

## Tuning
The model is learning, but we can do better. 

**At this point, you can play with the above code to change hyperparameters and note changes to your results.**

Alternatively, you can step through tuning this model below.

## Walk-through tuning:
Perhaps we did not have enough model parameters to accurately represent the mapping. **Remedy this by increasing the number of hidden neurons to 20.**

In [32]:
#Use the same code as in the previous cell
#simply change the number of neurons in the hidden layer

We see that we got little improvement here. Another hyperparameter to adjust is *batch size*, which is the number of training examples used to calculate the gradient on each step. While you may initially think that a higher batch size leads to faster or more accurate training, in practice this is not true. The "noise" that arises from using less training examples at each iteration can actually help find the global minimum of the loss function.
(See here for more info: https://arxiv.org/pdf/1609.04836.pdf)

Try decreasing the batch size to 16.

In [33]:
#Complete me:
results = model.fit(X, y, epochs=10, batch_size=16, validation_split=0.2)

Epoch 1/10
50/50 [==============================] - 0s 2ms/step - loss: 0.0902 - val_loss: 0.0537
Epoch 2/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0505 - val_loss: 0.0497
Epoch 3/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0497
Epoch 4/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 5/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 6/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 7/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 8/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 9/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 10/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497


This is starting to do better but has significant room for improvement.

Another hyperparameter to tune is the *learning rate*. 

 - If the learning rate is too high, we are taking too large of a step in the gradient descent at each iteration and will miss narrow minima in the loss function. 
 - If the learning rate is too small, then we are not traveling far enough in each iteration and we will take far too long to reach a minimum. 

Perhaps the learning rate is too high and the network can't fine tune. Try decreasing the learning rate to 0.001.

In [34]:
# Complete me:

optimize = tf.keras.optimizers.Adam(learning_rate=0.001) # Adam optimizer with a learning rate of 0.001
model.compile(optimizer=optimize,loss=loss_func) 
results = model.fit(X, y, epochs=10, batch_size=16, validation_split=0.2)

Epoch 1/10
50/50 [==============================] - 0s 3ms/step - loss: 0.0498 - val_loss: 0.0497
Epoch 2/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 3/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 4/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 5/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 6/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 7/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 8/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 9/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 10/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497


This is not really that much better, but now there is evidence of *overtraining* or *overfitting* -- the training loss is so much lower than the validation loss. 

A common fix to this is adding *dropout layers*. Try adding a dropout layer with dropout rate of 0.5. <https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout>

You can also try batch normalization: <https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization>

In [37]:
#Dropout layers are located under tf.keras.layers. 
#They take the dropout rate as their only argument.
#BatchNormalization layers are also under tf.keras.layers, and in the simplest use case, take no arguments

#Complete me:
model.add(tf.keras.layers.Dropout(0.5))
results = model.fit(X, y, epochs=10, batch_size=16, validation_split=0.2)

Epoch 1/10
50/50 [==============================] - 0s 2ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 2/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 3/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 4/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 5/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 6/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 7/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 8/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 9/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497
Epoch 10/10
50/50 [==============================] - 0s 1ms/step - loss: 0.0497 - val_loss: 0.0497


This clearly stopped the overtraining problem, but it still isn't training well. Now, try training on the full dataset with a more reasonable validation split of 0.2. Use a single hidden layer with 20 neurons, a learning rate of 0.001, and a batch size of 256. Just run it for 10 epochs.

In [40]:
#Complete me:

model = tf.keras.Sequential() #Define the model object
model.add(tf.keras.layers.Dense(units=20, # Number of neurons in hidden layer 
                                input_shape=(len(features),), 
                                activation="relu")
         ) #Add a single hidden layer
model.add(tf.keras.layers.Dense(units=1, activation="linear"))

optimize = tf.keras.optimizers.Adam(learning_rate=0.001) # Adam optimizer with a learning rate of 0.001
loss_func = tf.keras.losses.MeanSquaredError() # choosing MSE

# compile model
model.compile(optimizer=optimize,loss=loss_func) 

X_ = complete_data[features]
y_ = complete_data[masses.columns]

results = model.fit(X_, y_, epochs=10, batch_size=256, validation_split=0.2)

Epoch 1/10
113/113 [==============================] - 0s 2ms/step - loss: 3.9785 - val_loss: 0.8142
Epoch 2/10
113/113 [==============================] - 0s 1ms/step - loss: 0.5114 - val_loss: 0.3315
Epoch 3/10
113/113 [==============================] - 0s 1ms/step - loss: 0.2632 - val_loss: 0.2170
Epoch 4/10
113/113 [==============================] - 0s 1ms/step - loss: 0.1898 - val_loss: 0.1695
Epoch 5/10
113/113 [==============================] - 0s 1ms/step - loss: 0.1537 - val_loss: 0.1415
Epoch 6/10
113/113 [==============================] - 0s 1ms/step - loss: 0.1316 - val_loss: 0.1240
Epoch 7/10
113/113 [==============================] - 0s 1ms/step - loss: 0.1165 - val_loss: 0.1115
Epoch 8/10
113/113 [==============================] - 0s 1ms/step - loss: 0.1057 - val_loss: 0.1026
Epoch 9/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0976 - val_loss: 0.0953
Epoch 10/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0911 - val_loss: 0.0894

This clearly resulted in a significant improvement and shows how important having a large enough dataset is. Moving on to the choice in activation functions, ReLU is not the only available choice, although it is one of the most popular ones currently. Try training a network using a sigmoid or tanh activation function.

In [43]:
#Simply change relu to sigmoid or tanh to change the activation function

#Complete me:

model = tf.keras.Sequential() #Define the model object
model.add(tf.keras.layers.Dense(units=20, # Number of neurons in hidden layer 
                                input_shape=(len(features),), 
                                activation="tanh")
         ) #Add a single hidden layer
model.add(tf.keras.layers.Dense(units=1, activation="linear"))

optimize = tf.keras.optimizers.Adam(learning_rate=0.001) # Adam optimizer with a learning rate of 0.001
loss_func = tf.keras.losses.MeanSquaredError() # choosing MSE

# compile model
model.compile(optimizer=optimize,loss=loss_func) 

X_ = complete_data[features]
y_ = complete_data[masses.columns]

results = model.fit(X_, y_, epochs=10, batch_size=256, validation_split=0.2)

Epoch 1/10
113/113 [==============================] - 0s 2ms/step - loss: 0.4915 - val_loss: 0.1445
Epoch 2/10
113/113 [==============================] - 0s 1ms/step - loss: 0.1193 - val_loss: 0.1012
Epoch 3/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0903 - val_loss: 0.0838
Epoch 4/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0773 - val_loss: 0.0741
Epoch 5/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0699 - val_loss: 0.0680
Epoch 6/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0649 - val_loss: 0.0636
Epoch 7/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0613 - val_loss: 0.0605
Epoch 8/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0587 - val_loss: 0.0582
Epoch 9/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0567 - val_loss: 0.0564
Epoch 10/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0552 - val_loss: 0.0551

Next, try adding 2 new hidden layers to the network. Use the ReLU activation function.

In [46]:
#Complete me: COMPLETE MODEL
model = tf.keras.Sequential() #Define the model object
model.add(tf.keras.layers.Dense(units=20, # Number of neurons in hidden layer 
                                input_shape=(len(features),), 
                                activation="ReLU")
         ) #Add a single hidden layer 1
model.add(tf.keras.layers.Dense(units=20, # Number of neurons in hidden layer 
                                input_shape=(len(features),), 
                                activation="ReLU")
         ) #Add a single hidden layer 2
model.add(tf.keras.layers.Dense(units=20, # Number of neurons in hidden layer 
                                input_shape=(len(features),), 
                                activation="ReLU")
         ) #Add a single hidden layer 3
model.add(tf.keras.layers.Dense(units=1, activation="linear"))

optimize = tf.keras.optimizers.Adam(learning_rate=0.001) # Adam optimizer with a learning rate of 0.001
loss_func = tf.keras.losses.MeanSquaredError() # choosing MSE

# compile model
model.compile(optimizer=optimize,loss=loss_func) 

X_ = complete_data[features]
y_ = complete_data[masses.columns]

results = model.fit(X_, y_, epochs=10, batch_size=256, validation_split=0.2)

Epoch 1/10
113/113 [==============================] - 1s 2ms/step - loss: 0.4924 - val_loss: 0.0622
Epoch 2/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0578 - val_loss: 0.0549
Epoch 3/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0538 - val_loss: 0.0529
Epoch 4/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0524 - val_loss: 0.0520
Epoch 5/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0517 - val_loss: 0.0513
Epoch 6/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0511 - val_loss: 0.0509
Epoch 7/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0508 - val_loss: 0.0506
Epoch 8/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0505 - val_loss: 0.0505
Epoch 9/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0504 - val_loss: 0.0503
Epoch 10/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0503 - val_loss: 0.0502

Clearly, adding more layers helps improve the quality of the network. There is a limit to how effective this is though. Try having 5 hidden layers.

In [47]:
#Complete me: COMPLETE MODEL
model = tf.keras.Sequential() #Define the model object
model.add(tf.keras.layers.Dense(units=20, # Number of neurons in hidden layer 
                                input_shape=(len(features),), 
                                activation="ReLU")
         ) #Add a single hidden layer 1
model.add(tf.keras.layers.Dense(units=20, # Number of neurons in hidden layer 
                                input_shape=(len(features),), 
                                activation="ReLU")
         ) #Add a single hidden layer 2
model.add(tf.keras.layers.Dense(units=20, # Number of neurons in hidden layer 
                                input_shape=(len(features),), 
                                activation="ReLU")
         ) #Add a single hidden layer 3
model.add(tf.keras.layers.Dense(units=20, # Number of neurons in hidden layer 
                                input_shape=(len(features),), 
                                activation="ReLU")
         ) #Add a single hidden layer 4
model.add(tf.keras.layers.Dense(units=20, # Number of neurons in hidden layer 
                                input_shape=(len(features),), 
                                activation="ReLU")
         ) #Add a single hidden layer 5
model.add(tf.keras.layers.Dense(units=1, activation="linear"))

optimize = tf.keras.optimizers.Adam(learning_rate=0.001) # Adam optimizer with a learning rate of 0.001
loss_func = tf.keras.losses.MeanSquaredError() # choosing MSE

# compile model
model.compile(optimizer=optimize,loss=loss_func) 

X_ = complete_data[features]
y_ = complete_data[masses.columns]

results = model.fit(X_, y_, epochs=10, batch_size=256, validation_split=0.2)

Epoch 1/10
113/113 [==============================] - 1s 2ms/step - loss: 0.0555 - val_loss: 0.0507
Epoch 2/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0503 - val_loss: 0.0500
Epoch 3/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0499 - val_loss: 0.0499
Epoch 4/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0499 - val_loss: 0.0498
Epoch 5/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0498
Epoch 6/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0498
Epoch 7/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0498
Epoch 8/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0498
Epoch 9/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0498
Epoch 10/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0498

Now, see what happens when you increase the number of neurons per layer from 20 to 50 in the 3 hidden layer model. Consider how they perform compared to ReLU now. 

In [48]:
#Complete me:
#Complete me: COMPLETE MODEL
model = tf.keras.Sequential() #Define the model object
model.add(tf.keras.layers.Dense(units=20, # Number of neurons in hidden layer 
                                input_shape=(len(features),), 
                                activation="ReLU")
         ) #Add a single hidden layer 1
model.add(tf.keras.layers.Dense(units=35, # Number of neurons in hidden layer 
                                input_shape=(len(features),), 
                                activation="ReLU")
         ) #Add a single hidden layer 2
model.add(tf.keras.layers.Dense(units=50, # Number of neurons in hidden layer 
                                input_shape=(len(features),), 
                                activation="ReLU")
         ) #Add a single hidden layer 3
model.add(tf.keras.layers.Dense(units=1, activation="linear"))

optimize = tf.keras.optimizers.Adam(learning_rate=0.001) # Adam optimizer with a learning rate of 0.001
loss_func = tf.keras.losses.MeanSquaredError() # choosing MSE

# compile model
model.compile(optimizer=optimize,loss=loss_func) 

X_ = complete_data[features]
y_ = complete_data[masses.columns]

results = model.fit(X_, y_, epochs=10, batch_size=256, validation_split=0.2)

Epoch 1/10
113/113 [==============================] - 1s 2ms/step - loss: 0.1676 - val_loss: 0.0558
Epoch 2/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0538 - val_loss: 0.0527
Epoch 3/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0520 - val_loss: 0.0516
Epoch 4/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0512 - val_loss: 0.0510
Epoch 5/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0508 - val_loss: 0.0507
Epoch 6/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0505 - val_loss: 0.0505
Epoch 7/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0504 - val_loss: 0.0503
Epoch 8/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0502 - val_loss: 0.0502
Epoch 9/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0502 - val_loss: 0.0501
Epoch 10/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0501 - val_loss: 0.0501

Try using the sigmoid and the tanh activation functions again and compare them to ReLU.

In [49]:
#Complete me:
#Complete me:
#Complete me: COMPLETE MODEL
model = tf.keras.Sequential() #Define the model object
model.add(tf.keras.layers.Dense(units=20, # Number of neurons in hidden layer 
                                input_shape=(len(features),), 
                                activation="tanh")
         ) #Add a single hidden layer 1
model.add(tf.keras.layers.Dense(units=35, # Number of neurons in hidden layer 
                                input_shape=(len(features),), 
                                activation="tanh")
         ) #Add a single hidden layer 2
model.add(tf.keras.layers.Dense(units=50, # Number of neurons in hidden layer 
                                input_shape=(len(features),), 
                                activation="tanh")
         ) #Add a single hidden layer 3
model.add(tf.keras.layers.Dense(units=1, activation="linear"))

optimize = tf.keras.optimizers.Adam(learning_rate=0.001) # Adam optimizer with a learning rate of 0.001
loss_func = tf.keras.losses.MeanSquaredError() # choosing MSE

# compile model
model.compile(optimizer=optimize,loss=loss_func) 

X_ = complete_data[features]
y_ = complete_data[masses.columns]

results = model.fit(X_, y_, epochs=10, batch_size=256, validation_split=0.2)

Epoch 1/10
113/113 [==============================] - 1s 2ms/step - loss: 0.0616 - val_loss: 0.0506
Epoch 2/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0502 - val_loss: 0.0500
Epoch 3/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0500 - val_loss: 0.0499
Epoch 4/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0499 - val_loss: 0.0499
Epoch 5/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0498
Epoch 6/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0498
Epoch 7/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0498
Epoch 8/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0498
Epoch 9/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0498
Epoch 10/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0498

In [50]:
#Complete me:
#Complete me:
#Complete me: COMPLETE MODEL
model = tf.keras.Sequential() #Define the model object
model.add(tf.keras.layers.Dense(units=20, # Number of neurons in hidden layer 
                                input_shape=(len(features),), 
                                activation="sigmoid")
         ) #Add a single hidden layer 1
model.add(tf.keras.layers.Dense(units=35, # Number of neurons in hidden layer 
                                input_shape=(len(features),), 
                                activation="sigmoid")
         ) #Add a single hidden layer 2
model.add(tf.keras.layers.Dense(units=50, # Number of neurons in hidden layer 
                                input_shape=(len(features),), 
                                activation="sigmoid")
         ) #Add a single hidden layer 3
model.add(tf.keras.layers.Dense(units=1, activation="linear"))

optimize = tf.keras.optimizers.Adam(learning_rate=0.001) # Adam optimizer with a learning rate of 0.001
loss_func = tf.keras.losses.MeanSquaredError() # choosing MSE

# compile model
model.compile(optimizer=optimize,loss=loss_func) 

X_ = complete_data[features]
y_ = complete_data[masses.columns]

results = model.fit(X_, y_, epochs=10, batch_size=256, validation_split=0.2)

Epoch 1/10
113/113 [==============================] - 1s 2ms/step - loss: 0.0499 - val_loss: 0.0498
Epoch 2/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0498
Epoch 3/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0498
Epoch 4/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0498
Epoch 5/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0498
Epoch 6/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0498
Epoch 7/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0498
Epoch 8/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0498
Epoch 9/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0498
Epoch 10/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0498 - val_loss: 0.0498

This difference in performance, especially with the sigmoid function, is known as the vanishing gradient problem. If the value for any one the neurons gets too far away from 0, the gradient for sigmoid and tanh gets really close to 0. This means that **for deeper networks it is much more difficult to update the weights in the earlier layers as their gradient is so small**. Now, remove the fifth column from the input data, the charge, and see what happens when training. Why do you think including charge has such a large impact, even though it is not required to compute invariant mass?

In [56]:
#Complete me:
features_ = [i for i in features if not i.startswith('q')]


#Complete me:
#Complete me:
#Complete me: COMPLETE MODEL
model = tf.keras.Sequential() #Define the model object
model.add(tf.keras.layers.Dense(units=20, # Number of neurons in hidden layer 
                                input_shape=(len(features_),), 
                                activation="ReLU")
         ) #Add a single hidden layer 1
model.add(tf.keras.layers.Dense(units=35, # Number of neurons in hidden layer 
                                input_shape=(len(features_),), 
                                activation="ReLU")
         ) #Add a single hidden layer 2
model.add(tf.keras.layers.Dense(units=50, # Number of neurons in hidden layer 
                                input_shape=(len(features_),), 
                                activation="ReLU")
         ) #Add a single hidden layer 3
model.add(tf.keras.layers.Dense(units=1, activation="linear"))

optimize = tf.keras.optimizers.Adam(learning_rate=0.001) # Adam optimizer with a learning rate of 0.001
loss_func = tf.keras.losses.MeanSquaredError() # choosing MSE

# compile model
model.compile(optimizer=optimize,loss=loss_func) 

X_ = complete_data[features_]
y_ = complete_data[masses.columns]

results = model.fit(X_, y_, epochs=10, batch_size=256, validation_split=0.2)

Epoch 1/10
113/113 [==============================] - 1s 2ms/step - loss: 0.1326 - val_loss: 0.0584
Epoch 2/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0550 - val_loss: 0.0530
Epoch 3/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0522 - val_loss: 0.0516
Epoch 4/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0513 - val_loss: 0.0510
Epoch 5/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0509 - val_loss: 0.0507
Epoch 6/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0506 - val_loss: 0.0505
Epoch 7/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0504 - val_loss: 0.0503
Epoch 8/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0503 - val_loss: 0.0502
Epoch 9/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0502 - val_loss: 0.0502
Epoch 10/10
113/113 [==============================] - 0s 1ms/step - loss: 0.0501 - val_loss: 0.0501

Finally, there are other options for the loss function. Try experimenting with alternatives to mean squared error.

<https://www.tensorflow.org/api_docs/python/tf/keras/losses>

You can also try some other optimizers -- for example, sgd (with and without momentum), rmsprop, adagrad, adadelta, adamax, and nadam. <https://www.tensorflow.org/api_docs/python/tf/keras/optimizers>

